# 查询和对话

## 参数和方法定义

In [1]:
import io
import os
import boto3
import json
from opensearchpy import OpenSearch, RequestsHttpConnection
from sagemaker.huggingface.model import HuggingFacePredictor

from dotenv import load_dotenv
load_dotenv()

host = 'https://search-devax-search-bot-kcbw3pozuzdstr77ir44wzwmfy.us-east-1.es.amazonaws.com'
port = 443
region = 'us-east-1' 

llm_endpoint_name = "chatglm2-lmi-model"
vetctor_endpoint_name='text2vector'
index_name = 'doc_embeddings'

auth = (os.environ.get("AOS_USER"), os.environ.get("AOS_PWD")) 

smr = boto3.client('sagemaker-runtime')

parameters = {
  "max_length": 4092,
  "temperature": 0.01,
  "top_p":0.8
}

predictor_vector = HuggingFacePredictor(
  endpoint_name= vetctor_endpoint_name
)


class StreamScanner:
    def __init__(self):
        self.buff = io.BytesIO()
        self.read_pos = 0
        
    def write(self, content):
        self.buff.seek(0, io.SEEK_END)
        self.buff.write(content)
        
    def readlines(self):
        self.buff.seek(self.read_pos)
        for line in self.buff.readlines():
            if line[-1] != b'\n':
                self.read_pos += len(line)
                yield line[:-1]
                
    def reset(self):
        self.read_pos = 0


client = OpenSearch(
    hosts = [f'{host}:{port}'],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)


def search(sentense):
    sentence_vec = predictor_vector.predict({"text": sentense})
    query_body = {
        "size": 3,
        "query": {
            "knn": {
                "content_vec": {
                    "vector": sentence_vec,
                    "k": 2
                }
            }
        }
    }

    response = client.search(index=index_name, body=query_body)
    result =  map(map_res, response["hits"]["hits"])
    return list(result)

def map_res(obj):
    return {
        "id": obj["_id"],
        "score": obj["_score"],
        "text": obj["_source"]["text"],
        "origion": obj["_source"]["origion"],

    }

def ask(q):
  answers = search(q)
  strAnswers = ""
  references = {}
  for an in answers:
      strAnswers += an['text'] + "\n\n"
      origion = an['origion']
      if origion in references :
        references[origion] += 1
      else:
         references[origion] = 1
  ori_prompt = f"""
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{strAnswers}

Question: {q}
            
Helpful Answer:

  """
  # print("Prompt: ")
  # print(ori_prompt)

  print("问题: ", q)


  response_model = smr.invoke_endpoint_with_response_stream(
              EndpointName=llm_endpoint_name,
              Body=json.dumps(
              {
                  "inputs": ori_prompt,
                  "parameters": parameters,
                  "history" : []
              }
              ),
              ContentType="application/json",
          )

  event_stream = response_model['Body']
  # print(event_stream)
  scanner = StreamScanner()
  for event in event_stream:
      scanner.write(event['PayloadPart']['Bytes'])
      for line in scanner.readlines():
          try:
              resp = json.loads(line)
              print(resp.get("outputs")['outputs'], end='')
          except Exception as e:
              continue
  print()
  print("-"*20)
  print(references)
  

## 测试

In [4]:

search("推荐一款电动车防雨罩，并告诉我价格") #  碳纤维笔筒

[{'id': 'wtHffYoBGnkxYaTOIXTE',
  'score': 0.2580855,
  'text': '电动车防雨罩防晒罩雨棚蓬摩托车电瓶车防水遮雨车衣车罩四季通用 加厚铝膜【蓝色】 S无耳加厚【适用1.6米内】\n商品毛重：1.0kg 货号：11-4-0.4-BDZ-2类型：半罩材质：其他适用车型：车型通用\n价格 ￥43.6',
  'origion': './docs_ec/g5.txt'},
 {'id': 'xsTffYoBIvxDUYGoGYYL',
  'score': 0.20946097,
  'text': '雅迪新国标电动车座套防雨防晒新款皮革爱玛电车坐垫套防水通用型 【四面弹皮革】圆头 单个前座套\n商品名称：雅迪新国标电动车座套防雨防晒新款皮革爱玛电车坐垫套防水通用型 【四面弹皮革】圆头 类别：平面款表层主材质：皮功能：加热下单需绑定车型：需绑定车型适用季节：四季通用适用场景：商务接待套件类别：前排单座\n价格 ￥11.72',
  'origion': './docs_ec/g4.txt'},
 {'id': 'xcTffYoBIvxDUYGoBYbL',
  'score': 0.11006618,
  'text': 'Kindle Oasis Essentials Bundle including Kindle Oasis (Graphite, Ad-Supported), Amazon Leather Cover, and Power Adapter\nIncludes the latest Kindle Oasis with Special Offers, 32 GB, Wi-Fi, Graphite, Amazon Leather Cover (Merlot), and Power Adapter.\nOur best 7", 300ppi flush-front Paperwhite display.\nAdjustable warm light to shift screen shade from white to amber.\nWaterproof (IPX8) so you can read in the bath or by the pool.\nThin and light ergono

In [5]:

ask("推荐一款电动车防雨罩，并告诉我价格")

问题:  推荐一款电动车防雨罩，并告诉我价格
电动车防雨罩。根据提供的信息，我推荐您使用“电动车防雨罩防晒罩雨棚蓬摩托车电瓶车防水遮雨车衣车罩四季通用加厚铝膜【蓝色】”。这款产品具有以下特点：

1. 适用车型：车型通用。
2. 材质：其他材质。
3. 颜色：蓝色。
4. 货号：11-4-0.4-BDZ-2。
5. 商品毛重：1.0kg。
6. 功能：防雨、防晒、遮阳。

这款产品可以帮助您保护电动车免受雨淋和防晒，确保您的电动车在四季都能够保持干燥。价格方面，该产品的售价为人民币43.6元。
--------------------
{'./docs_ec/g5.txt': 1, './docs_ec/g4.txt': 1, './docs_ec/g3.txt': 1}


In [6]:
ask("Features and price of carbon fiber pen holder") 

问题:  Features and price of carbon fiber pen holder
 carbon fiber pen holder features a handmade linear carbon fiber and a无铅黄铜 design. It has a length of 180mm and an outer diameter of 16mm, and an inner diameter of 13mm. The pen holder has a weight of 30g and is produced in Italy. It comes with an original factory-sealed packaging and a user manual. The price of the carbon fiber pen holder is ￥728.
--------------------
{'./docs_ec/g1.txt': 1, './docs_ec/g5.txt': 1, './docs_ec/g3.txt': 1}


In [7]:
ask("中国最好的大学是哪个？") 

问题:  中国最好的大学是哪个？
，我是一个人工智能助手，无法访问互联网获取信息。我的知识中没有关于中国最好的大学的信息，因此无法回答这个问题。
--------------------
{'./docs_ec/g1.txt': 1, './docs_ec/g4.txt': 1, './docs_ec/g2.txt': 1}
